# ChipChat Tutorial – Example B
## Example: Sequence Detector FSM (1011)

Course: LLM4ChipDesign  
Instructor: Ramesh Karri, Weihua Xiao  

This notebook demonstrates:
- Prompt engineering for sequential RTL (FSM)
- LLM-generated Verilog cleanup
- Verification using a self-checking testbench
- Reproducible RTL generation in Google Colab

### Overview
This notebook solves the **sequence_detector FSM** tutorial example from
the ChipChat assignment. It follows the Part I requirements by:
- Defining a precise hardware prompt
- Capturing raw LLM output
- Cleaning and extracting synthesizable Verilog
- Verifying correctness using simulation
- Ensuring full reproducibility in Google Colab

In [15]:
!apt-get update
!apt-get install -y iverilog


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
iverilog is already the newest version (11.0-1.1).
0 upgraded, 0 newly installed, 0 to remove 

In [16]:
!pip install --quiet openai

In [17]:
import os
from openai import OpenAI

In [18]:
os.environ["OPENAI_API_KEY"] = " "
client = OpenAI()

In [19]:
prompt = """
Generate synthesizable Verilog-2012 for a sequence detector FSM.

Requirements:
- Detect the input sequence 1011
- Overlapping sequences allowed
- Inputs: clk, reset, in_bit
- Output: detected (asserted for one clock cycle)
- Synchronous design with active-high reset
- Use enum for FSM states
- No delays, no latches, no behavioral constructs
- Must compile with: iverilog -g2012
"""
print(prompt)


Generate synthesizable Verilog-2012 for a sequence detector FSM.

Requirements:
- Detect the input sequence 1011
- Overlapping sequences allowed
- Inputs: clk, reset, in_bit
- Output: detected (asserted for one clock cycle)
- Synchronous design with active-high reset
- Use enum for FSM states
- No delays, no latches, no behavioral constructs
- Must compile with: iverilog -g2012



In [20]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

raw_output = response.choices[0].message.content
print(raw_output)

Below is a synthesizable Verilog-2012 code for a sequence detector FSM that detects the input sequence `1011`. The design meets all the specified requirements, including the use of enumerated types for FSM states, synchronous design with an active-high reset, and no behavioral constructs or delays.

```verilog
module sequence_detector (
    input wire clk,
    input wire reset,
    input wire in_bit,
    output reg detected
);

    // Define the states using an enumerated type
    typedef enum logic [2:0] {
        S0 = 3'b000, // Initial state
        S1 = 3'b001, // State after receiving '1'
        S2 = 3'b010, // State after receiving '10'
        S3 = 3'b011, // State after receiving '101'
        S4 = 3'b100  // State after receiving '1011'
    } state_t;

    state_t current_state, next_state;

    // State transition logic
    always_ff @(posedge clk or posedge reset) begin
        if (reset) begin
            current_state <= S0;
            detected <= 1'b0;
        end else 

In [21]:
verilog_code = """
module sequence_detector (
    input  logic clk,
    input  logic reset,
    input  logic in_bit,
    output logic detected
);

    typedef enum logic [2:0] {
        IDLE,
        S1,
        S10,
        S101
    } state_t;

    state_t state, next_state;

    // State register
    always_ff @(posedge clk) begin
        if (reset)
            state <= IDLE;
        else
            state <= next_state;
    end

    // Registered output
    always_ff @(posedge clk) begin
        if (reset)
            detected <= 1'b0;
        else
            detected <= (state == S101 && in_bit);
    end

    // Next-state logic
    always_comb begin
        next_state = state;

        case (state)
            IDLE:  if (in_bit) next_state = S1;
            S1:    if (!in_bit) next_state = S10;
                   else next_state = S1;
            S10:   if (in_bit) next_state = S101;
                   else next_state = IDLE;
            S101:  if (in_bit) next_state = S1;
                   else next_state = S10;
        endcase
    end

endmodule
"""
print(verilog_code)


module sequence_detector (
    input  logic clk,
    input  logic reset,
    input  logic in_bit,
    output logic detected
);

    typedef enum logic [2:0] {
        IDLE,
        S1,
        S10,
        S101
    } state_t;

    state_t state, next_state;

    // State register
    always_ff @(posedge clk) begin
        if (reset)
            state <= IDLE;
        else
            state <= next_state;
    end

    // Registered output
    always_ff @(posedge clk) begin
        if (reset)
            detected <= 1'b0;
        else
            detected <= (state == S101 && in_bit);
    end

    // Next-state logic
    always_comb begin
        next_state = state;

        case (state)
            IDLE:  if (in_bit) next_state = S1;
            S1:    if (!in_bit) next_state = S10;
                   else next_state = S1;
            S10:   if (in_bit) next_state = S101;
                   else next_state = IDLE;
            S101:  if (in_bit) next_state = S1;
                   else 

In [22]:
with open("sequence_detector.v", "w") as f:
    f.write(verilog_code)

In [23]:
tb_code = """
`timescale 1ns/1ps
module tb_sequence_detector;

    logic clk, reset, in_bit;
    logic detected;

    sequence_detector dut (
        .clk(clk),
        .reset(reset),
        .in_bit(in_bit),
        .detected(detected)
    );

    always #5 clk = ~clk;

    initial begin
        clk = 0;
        reset = 1;
        in_bit = 0;
        #12 reset = 0;

        // Apply sequence: 1011
        #10 in_bit = 1;
        #10 in_bit = 0;
        #10 in_bit = 1;
        #10 in_bit = 1;

        #10;
        if (!detected) begin
            $display("FAIL: sequence not detected");
            $finish;
        end

        $display("All tests passed!");
        $finish;
    end

endmodule
"""

In [24]:
with open("tb_sequence_detector.v", "w") as f:
    f.write(tb_code)

In [25]:
!ls -l *.v

-rw-r--r-- 1 root root 1053 Feb  8 15:53 sequence_detector.v
-rw-r--r-- 1 root root  690 Feb  8 15:53 tb_sequence_detector.v


In [26]:
!iverilog -g2012 -o simv sequence_detector.v tb_sequence_detector.v

### Verification
The FSM is verified using a self-checking testbench consistent with the
ChipChat tutorial style. The testbench applies the target input sequence and
checks that the `detected` output is asserted correctly.

In [27]:
!vvp simv

All tests passed!
